In [6]:
import torch
from torch_geometric.data import Data
import json

# Create PyTorch Geometric Data object

In [7]:
def create_data_object(edge_index, relations, entity_to_idx):
    unique_relations = list(set(relations))
    relation_mapping = {relation: index for index, relation in enumerate(unique_relations)}

    edge_index = torch.tensor(edge_index).t().contiguous()
    edge_attr = torch.tensor([relation_mapping[rel] for rel in relations])

    return Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(entity_to_idx))


def load_data_json(filename='../Datasets/MetaQA_dataset/processed/idxes.json'):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data['entity_to_idx'], data['edge_index'], data['relations']

In [8]:
loaded_entity_to_idx, loaded_edge_index, loaded_relations = load_data_json() 
data = create_data_object(loaded_edge_index, loaded_relations, loaded_entity_to_idx)
print(data)

Data(edge_index=[2, 133582], edge_attr=[133582], num_nodes=43234)
